# Non-Maximum Supression Validation


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from plantclef.spark import get_spark

spark = get_spark(cores=4)
display(spark)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/26 19:29:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/26 19:29:57 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
25/04/26 19:29:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
import os
from pathlib import Path

# Get list of stored filed in cloud bucket
root = Path(os.path.expanduser("~"))
! date

Sat Apr 26 07:30:00 PM EDT 2025


### NMS detections

In [5]:
# Path and dataset names
data_path = f"{root}/p-dsgt_clef2025-0/shared/plantclef/data"
test_path = f"{data_path}/detection/test_2024_subset20/test_2024_subset20_detection_v1"
detection_df = spark.read.parquet(test_path)
detection_df.printSchema()
detection_df.show(n=5)

root
 |-- image_name: string (nullable = true)
 |-- output: struct (nullable = true)
 |    |-- extracted_bbox: array (nullable = true)
 |    |    |-- element: binary (containsNull = true)
 |    |-- boxes: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |-- scores: array (nullable = true)
 |    |    |-- element: float (containsNull = true)
 |    |-- text_labels: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- sample_id: integer (nullable = true)



+--------------------+--------------------+---------+
|          image_name|              output|sample_id|
+--------------------+--------------------+---------+
|CBN-PdlC-C1-20200...|{[[89 50 4E 47 0D...|        0|
|CBN-PdlC-B6-20150...|{[[89 50 4E 47 0D...|        0|
|CBN-Pla-B2-201407...|{[[89 50 4E 47 0D...|        0|
|CBN-Pla-D5-201607...|{[[89 50 4E 47 0D...|        0|
|CBN-PdlC-A1-20180...|{[[89 50 4E 47 0D...|        0|
+--------------------+--------------------+---------+
only showing top 5 rows



In [6]:
detection_df.count()

20